# Creates SBML Models for Sub-SBML
Note: SBML for Bistable Switch already created in Adding rR2 notebook. File name is BiSwitch_CRN.xml

1. Simple rR1 diffusion model (Agrima's atc model)
1. Advanced diffusion of rR1 (Agrima's dna export model)
1. External source with rI# (try integration)

# Bistable Switch
SBML file down below

In [1]:
from biocrnpyler import *
from genelet import *
import pylab as plt
import numpy as np

In [2]:
# Create CRN for bistable switch
Core1_off = Species ("Core1")
Core2_off = Species ("Core2")

Core1 = Genelet(Core1_off, transcript = "rR1", activator = "dA2", inhibitor = "rR2" )
Core2 = Genelet(Core2_off, transcript = "rR2", activator = "dA1", inhibitor = "rR1" )

M_BiSwitch = Mixture(name = "Switch_test", components = [Core1, Core2], parameter_file = "default_parameters.txt")
repr(M_BiSwitch)
CRN_BiSwitch = M_BiSwitch.compile_crn()

# Write SBML file
CRN_BiSwitch.write_sbml_file('BiSwitch_CRN.xml')

print(CRN_BiSwitch.pretty_print())

Species (18) = {0. Core1_OFF, 1. complex[dna[dA2]:Core1_OFF], 2. rna[rR1], 3. dna[dA2], 4. rna[rR2], 5. protein[RNAP], 6. protein[RNAseH], 7. complex[rna[rR2]:dna[dA2]], 8. complex[complex[dna[dA2]:Core1_OFF]:protein[RNAP]], 9. complex[Core1_OFF:protein[RNAP]], 10. complex[complex[rna[rR2]:dna[dA2]]:protein[RNAseH]], 11. Core2_OFF, 12. complex[dna[dA1]:Core2_OFF], 13. dna[dA1], 14. complex[rna[rR1]:dna[dA1]], 15. complex[complex[dna[dA1]:Core2_OFF]:protein[RNAP]], 16. complex[Core2_OFF:protein[RNAP]], 17. complex[complex[rna[rR1]:dna[dA1]]:protein[RNAseH]]}
Reactions (18) = [
0. Core1_OFF + dna[dA2] --> complex[dna[dA2]:Core1_OFF]        
        massaction: k_f(Core1_OFF,dna[dA2])=0.00394*Core1_OFF*dna[dA2]
1. complex[dna[dA2]:Core1_OFF] + rna[rR2] --> Core1_OFF + complex[rna[rR2]:dna[dA2]]        
        massaction: k_f(complex[dna[dA2]:Core1_OFF],rna[rR2])=0.0696*complex[dna[dA2]:Core1_OFF]*rna[rR2]
2. dna[dA2] + rna[rR2] --> complex[rna[rR2]:dna[dA2]]        
        massaction: k

# Produce 2N
SBML file below with DFHBI in separate reaction

In [3]:
# Create CRN of Produce 2N
Prod2_off = Species("Prod2")
Broccoli_Aptamer = Species("BrocApt")

Prod2 = Genelet(Prod2_off, transcript= Broccoli_Aptamer , activator= "dA1"  , inhibitor= "rR1")
M_Prod2 = Mixture(name = "Produce2_test", components = [Prod2], parameter_file = "default_parameters.txt")
repr(M_Prod2)
CRN_Prod2 = M_Prod2.compile_crn()

# Write SBML
CRN_Prod2.write_sbml_file('Prod2_CRN.xml')

print(CRN_Prod2.pretty_print())

Species (11) = {0. Prod2_OFF, 1. complex[Prod2_OFF:dna[dA1]], 2. BrocApt, 3. dna[dA1], 4. rna[rR1], 5. protein[RNAP], 6. protein[RNAseH], 7. complex[rna[rR1]:dna[dA1]], 8. complex[complex[Prod2_OFF:dna[dA1]]:protein[RNAP]], 9. complex[Prod2_OFF:protein[RNAP]], 10. complex[complex[rna[rR1]:dna[dA1]]:protein[RNAseH]]}
Reactions (9) = [
0. Prod2_OFF + dna[dA1] --> complex[Prod2_OFF:dna[dA1]]        
        massaction: k_f(Prod2_OFF,dna[dA1])=0.00394*Prod2_OFF*dna[dA1]
1. complex[Prod2_OFF:dna[dA1]] + rna[rR1] --> Prod2_OFF + complex[rna[rR1]:dna[dA1]]        
        massaction: k_f(complex[Prod2_OFF:dna[dA1]],rna[rR1])=0.0696*complex[Prod2_OFF:dna[dA1]]*rna[rR1]
2. dna[dA1] + rna[rR1] --> complex[rna[rR1]:dna[dA1]]        
        massaction: k_f(dna[dA1],rna[rR1])=0.0696*dna[dA1]*rna[rR1]
3. complex[Prod2_OFF:dna[dA1]] + protein[RNAP] <--> complex[complex[Prod2_OFF:dna[dA1]]:protein[RNAP]]        
        massaction: k_f(complex[Prod2_OFF:dna[dA1]],protein[RNAP])=38.857142857142854*com

#### Create CRN for DFHBI and Broc Apt Binding

In [4]:
### Species ###
Broccoli_Aptamer = Species("BrocApt")
DFHBI = Species("DFHBI")
Flu1 = ComplexSpecies([Broccoli_Aptamer,DFHBI])
species = [Broccoli_Aptamer, DFHBI, Flu1]

### Create Reaction ###
rxn1 = Reaction([Broccoli_Aptamer, DFHBI], [Flu1], propensity_type = "massaction", k= 9.96e-5)
rxns = [rxn1]

D = ChemicalReactionNetwork(species = species, reactions = rxns)
_ = D.write_sbml_file('DFHBI.xml')
print(D)

Species = BrocApt, DFHBI, complex_BrocApt_DFHBI
Reactions = [
	BrocApt + DFHBI --> complex_BrocApt_DFHBI        massaction: k_f(BrocApt,DFHBI)=9.96e-05*BrocApt*DFHBI
]


# rR1 external reservoir

In [5]:
##### Creating rR1 reservoir model####

from subsbml import createBasicSubsystem, SimpleModel


rR1_ss, rR1_model = createBasicSubsystem('external_reservoir')

per_second = rR1_model.createNewUnitDefinition(uId = 'per_second',uKind = libsbml.UNIT_KIND_SECOND,
                                                uExponent = -1)
substance = rR1_model.createNewUnitDefinition('substance',libsbml.UNIT_KIND_DIMENSIONLESS, uExponent = 1)

rR1_model.createNewSpecies('rna_rR1_e',sComp = 'external_reservoir', ListOfAmounts = 0, 
                             sConstant = False, sSubstance = 'substance')

# Write to SBML file 
exsub = rR1_ss.writeSBML('rR1_external_reservoir.xml')
print(rR1_ss)

# rR1 simple membrane model

rR1_internal $\leftrightarrow$ rR1_external

In [6]:
##### Creating a basic membrane model for IPTG transport####

from subsbml import createNewSubsystem, SimpleModel, SimpleReaction
import libsbml

rR1_mb_ss = createNewSubsystem()
model = rR1_mb_ss.createNewModel('rR1_membrane','second','mole','substance')

rR1_mb = SimpleModel(model)

per_second = rR1_mb.createNewUnitDefinition('per_second',libsbml.UNIT_KIND_SECOND,-1,0,1)
substance = rR1_mb.createNewUnitDefinition('substance',libsbml.UNIT_KIND_DIMENSIONLESS, 1, 0, 1)

################# Two important lines of code when creating membrane #########################
rR1_mb.createNewCompartment('internal','internal',1e-6,'litre',True)
rR1_mb.createNewCompartment('external','external',1e-6,'litre',True)
##############################################################################################

rR1_mb.createNewSpecies( 'rna_rR1_e','external',1e4,False,'substance')
rR1_mb.createNewSpecies( 'rna_rR1','internal',0,False,'substance')

rR1_mb.createNewParameter( 'kb',2e-3,False,'per_second')
rR1_mb.createNewParameter( 'kd',1e-5,False,'per_second')

# model = IPTG_mb_ss.getSBMLDocument().getModel()
r1_sbml = model.createReaction()
r1 = SimpleReaction(r1_sbml)
r1.setId('r1')
r1.setReversible(True)
r1.createNewReactant('rna_rR1_e',False,1)
r1.createNewProduct('rna_rR1', False, 1)
r1.createRate('kb * rna_rR1_e - kd * rna_rR1')

# Write to SBML file 
_ = rR1_mb_ss.writeSBML('simplemembrane_rR1.xml')
print('Model written to a SBML file successfully')

Model written to a SBML file successfully


# rR1 advanced membrane model (SID)

In [7]:
##### Creating a advanced membrane model for RNA transport####
##### Based on Agrima's DNA membrane 1 #####

from subsbml import createNewSubsystem, SimpleModel, SimpleReaction

rnaexp_mb_ss = createNewSubsystem()
model = rnaexp_mb_ss.createNewModel('rnaexp_membrane','second','mole','substance')

rnaexp_mb = SimpleModel(model)

per_second = rnaexp_mb.createNewUnitDefinition('per_second',libsbml.UNIT_KIND_SECOND,-1,0,1)
substance = rnaexp_mb.createNewUnitDefinition('substance',libsbml.UNIT_KIND_DIMENSIONLESS, 1, 0, 1)

################# Two important lines of code when creating membrane #########################
rnaexp_mb.createNewCompartment('internal','internal',1e-6,'litre',True)
rnaexp_mb.createNewCompartment('external','external',1e-6,'litre',True)
##############################################################################################

rnaexp_mb.createNewSpecies( 'rna_rR1_e','external',0,False,'substance')
rnaexp_mb.createNewSpecies( 'rna_rR1','internal',0,False,'substance')
rnaexp_mb.createNewSpecies( 'SID_b','internal',0,False,'substance')
rnaexp_mb.createNewSpecies( 'SID_b_rna_rR1','internal',0,False,'substance')

rnaexp_mb.createNewParameter( 'kb1',0.002,False,'per_second')
rnaexp_mb.createNewParameter( 'kd1',0.00001,False,'per_second')
rnaexp_mb.createNewParameter( 'kd2',0.01,False,'per_second')


model = rnaexp_mb_ss.getSBMLDocument().getModel()

R1 = SimpleReaction(model.createReaction())
R1.setId('R1')
R1.setReversible(True)
R1.createNewReactant('SID_b',False,1)
R1.createNewReactant('rna_rR1', False, 1)
R1.createNewProduct('SID_b_rna_rR1', False, 1)
R1.createRate('kb1 * SID_b * rna_rR1 - kd1 * SID_b_rna_rR1')


R2 = SimpleReaction(model.createReaction())
R2.setId('R2')
R2.setReversible(False)
R2.createNewReactant('SID_b_rna_rR1',False,1)
R2.createNewProduct('rna_rR1_e',False,1)
R2.createNewProduct('SID_b',False,1)
R2.createRate('kd2 * SID_b_rna_rR1')


# Write to SBML file 
_ = rnaexp_mb_ss.writeSBML('RNAexp_membrane1.xml')
print('Model written to a SBML file successfully')

Model written to a SBML file successfully


In [8]:
#### Agrima's dnaexp_membrane3.xml ####

from subsbml import createNewSubsystem, SimpleModel, SimpleReaction
import libsbml

rnaexp_mb_ss = createNewSubsystem()


model = rnaexp_mb_ss.createNewModel('rnaexp_membrane','second','mole','substance')

rnaexp_mb = SimpleModel(model)

per_second = rnaexp_mb.createNewUnitDefinition('per_second',libsbml.UNIT_KIND_SECOND,-1,0,1)
substance = rnaexp_mb.createNewUnitDefinition('substance',libsbml.UNIT_KIND_DIMENSIONLESS, 1, 0, 1)

################# Two important lines of code when creating membrane #########################
rnaexp_mb.createNewCompartment('internal','internal',1e-6,'litre',True)
rnaexp_mb.createNewCompartment('external','external',1e-6,'litre',True)
##############################################################################################

rnaexp_mb.createNewSpecies( 'rna_rR1_e','external',0,False,'substance')
rnaexp_mb.createNewSpecies( 'rna_rR1','internal',0,False,'substance')
rnaexp_mb.createNewSpecies( 'sid','internal',1,False,'substance')
rnaexp_mb.createNewSpecies( 'rna_rR1_sid','internal',0,False,'substance')

rnaexp_mb.createNewParameter( 'kb',0.2,False,'per_second') #0.2
rnaexp_mb.createNewParameter( 'kd',0.01,False,'per_second') #0.01

model = rnaexp_mb_ss.getSBMLDocument().getModel()

R1 = SimpleReaction(model.createReaction())
R1.setId('R1')
R1.setReversible(True)
R1.createNewReactant('sid',False,1)
R1.createNewReactant('rna_rR1', False, 1)
R1.createNewProduct('rna_rR1_sid', False, 1)
R1.createRate('kb * sid * rna_rR1 - kd * rna_rR1_sid')

R2 = SimpleReaction(model.createReaction())
R2.setId('R2')
R2.setReversible(True)
R2.createNewReactant('rna_rR1_sid',False,1)
R2.createNewProduct('rna_rR1_e',False,1)
R2.createNewProduct('sid',False,1)
R2.createRate('kd * rna_rR1_sid - kb * sid * rna_rR1_e')


# Write to SBML file 
_ = rnaexp_mb_ss.writeSBML('rR1_membrane1_detailed.xml')
print('Model written to a SBML file successfully')


Model written to a SBML file successfully
